In [5]:
import os
import tensorflow as tf
import numpy as np

In [9]:
TRAIN_PATH = "../data_train/processed_audio/train.npz"
TEST_PATH = "../data_train/processed_audio/test.npz"

data_train = np.load(TRAIN_PATH)
data_test = np.load(TEST_PATH)

X_train, y_train = data_train["X"], data_train["y"]
X_test, y_test = data_test["X"], data_test["y"]

BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = (
    tf.data.Dataset.from_tensor_slices((tf.cast(X_train, tf.float32), y_train))
    .cache()
    .shuffle(buffer_size=len(X_train))
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset.from_tensor_slices((tf.cast(X_test, tf.float32), y_test))
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

# Compresión de modelos a Tensorflow Lite

## Compresión y cuantización a entero de 8 bits

In [21]:
# Rutas
MODEL_H5_PATH = "../models/not_compressed_audio/model_CNNA.h5"
TFLITE_PATH = "../models/compressed_int8/model_CNNA.tflite"


# Dataset representativo para calibración de int8
def representative_dataset():
    for i in range(100):
        sample = X_train[i].astype(np.float32)[np.newaxis, :]
        yield [sample]

# Cargar el modelo H5 entrenado
model = tf.keras.models.load_model(MODEL_H5_PATH)

# Prepara el convertidor TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.representative_dataset = representative_dataset

# Convierte y guarda
tflite_quant = converter.convert()
with open(TFLITE_PATH, "wb") as f:
    f.write(tflite_quant)

print("Modelo TFLite guardado en:", TFLITE_PATH)

INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmpghu8ejoh\assets


INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmpghu8ejoh\assets
c:\Users\Jaime\anaconda3\envs\tf_env\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Modelo TFLite guardado en: ../models/compressed_int8/model_CNNA.tflite


Script para ver las operaciones  que realiza  el modelo

In [12]:
TFLITE_PATH = '../models/compressed_int8/model_NN.tflite'

interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
interpreter.allocate_tensors()

# Verificar tipo de entrada/salida
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Tipo de entrada: {input_details[0]['dtype']} ")
print(f"Tipo de salida: {output_details[0]['dtype']} ")

# Verificar operaciones utilizadas
ops_set = set()
for idx in range(len(interpreter._get_ops_details())):
    op_name = interpreter._get_ops_details()[idx]['op_name']
    ops_set.add(op_name)

print("\nOperaciones utilizadas en el modelo TFLite:")
for op in sorted(ops_set):
    print(f"{op}")


Tipo de entrada: <class 'numpy.int8'> 
Tipo de salida: <class 'numpy.int8'> 

Operaciones utilizadas en el modelo TFLite:
FULLY_CONNECTED
SOFTMAX


Script para pasa  los modelos  a un archivo binario

In [ ]:
# Rutas
INT8_TFLITE_PATH = "../models/compressed_int8/model_NN.tflite"
CPP_PATH = "../V_int8/main/model.cpp"
HEADER_PATH = "../V_int8/main/model.hpp"

# Cargar modelo TFLite cuantizado
with open(INT8_TFLITE_PATH, "rb") as f:
    tflite_model = f.read()

#Convertir a array de bytes en formato C++
cpp_array = ", ".join(str(b) for b in tflite_model)

# 📄 Generar código C++ (`model.cpp`)
cpp_code = f"""#include "model.hpp"

alignas(8) const unsigned char model_tflite[] = {{
    {cpp_array}
}};

const int model_tflite_len = {len(tflite_model)};
"""

# Guardar `model.cpp`
with open(CPP_PATH, "w") as f:
    f.write(cpp_code)

# 📄 Generar código C++ (`model.hpp`)
header_code = """#ifndef MODEL_H_
#define MODEL_H_

extern const unsigned char model_tflite[];
extern const int model_tflite_len;

#endif  // MODEL_H_
"""

# Guardar `model.h`
with open(HEADER_PATH, "w") as f:
    f.write(header_code)

print(f"Modelo exportado a C++ en {CPP_PATH}")
print(f"Header guardado en {HEADER_PATH}")

Modelo exportado a C++ en ../V_int8/main/model.cpp
Header guardado en ../V_int8/main/model.hpp


## Conversión a TFLite sin cuantizar

In [41]:
MODEL_H5_PATH = "../models/not_compressed_audio/model_CNNA.h5"
FLOAT_TFLITE_PATH = "../models/compressed_float32/model_CNNA.tflite"

model = tf.keras.models.load_model(MODEL_H5_PATH)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
tflite_model = converter.convert()
with open(FLOAT_TFLITE_PATH, "wb") as f:
    f.write(tflite_model)

print(f"Modelo cuantizado")

INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmp7owq2o4s\assets


INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmp7owq2o4s\assets


Modelo cuantizado


In [37]:
TFLITE_PATH = '../models/compressed_float32/model_CNN.tflite'

interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Tipo de entrada: {input_details[0]['dtype']} ")
print(f"Tipo de salida: {output_details[0]['dtype']} ")

ops_set = set()
for idx in range(len(interpreter._get_ops_details())):
    op_name = interpreter._get_ops_details()[idx]['op_name']
    ops_set.add(op_name)

print("\nOperaciones utilizadas en el modelo TFLite:")
for op in sorted(ops_set):
    print(f"{op}")


Tipo de entrada: <class 'numpy.float32'> 
Tipo de salida: <class 'numpy.float32'> 

Operaciones utilizadas en el modelo TFLite:
CONV_2D
EXPAND_DIMS
FULLY_CONNECTED
MAX_POOL_2D
PACK
RESHAPE
SHAPE
SOFTMAX
STRIDED_SLICE


In [42]:
# Rutas
FLOAT_TFLITE_PATH = "../models/compressed_float32/model_CNNA.tflite"
CPP_PATH = "../V_float/main/model.cpp"
HEADER_PATH = "../V_float/main/model.hpp"

# Cargar modelo TFLite cuantizado
with open(FLOAT_TFLITE_PATH, "rb") as f:
    tflite_model = f.read()

#Convertir a array de bytes en formato C++
cpp_array = ", ".join(str(b) for b in tflite_model)

# 📄 Generar código C++ (`model.cpp`)
cpp_code = f"""#include "model.hpp"

alignas(8) const unsigned char model_tflite[] = {{
    {cpp_array}
}};

const int model_tflite_len = {len(tflite_model)};
"""

# Guardar `model.cpp`
with open(CPP_PATH, "w") as f:
    f.write(cpp_code)

# 📄 Generar código C++ (`model.h`)
header_code = """#ifndef MODEL_H_
#define MODEL_H_

extern const unsigned char model_tflite[];
extern const int model_tflite_len;

#endif  // MODEL_H_
"""

# Guardar `model.h`
with open(HEADER_PATH, "w") as f:
    f.write(header_code)

print(f"Modelo exportado a C++ en {CPP_PATH}")
print(f"Header guardado en {HEADER_PATH}")

Modelo exportado a C++ en ../V_float/main/model.cpp
Header guardado en ../V_float/main/model.hpp


## Versiones MFCC


In [43]:
# Rutas de los datos procesados
TRAIN_PATH = "../data_train/processed_MFCC/train.npz"
TEST_PATH = "../data_train/processed_MFCC/test.npz"

# Cargar datos
data_train = np.load(TRAIN_PATH)
data_test = np.load(TEST_PATH)

X_train, y_train = data_train["X"], data_train["y"]
X_test, y_test = data_test["X"], data_test["y"]

In [44]:
MODEL_H5_PATH = "../models/not_compressed_MFCC/model_CNN.h5"
TFLITE_PATH = "../models/compressed_MFCC/model_CNN.tflite"

def representative_data_gen():
    # Definir el array de muestra con forma (49, 13) utilizando los datos de cada frame:
    sample_float32 = np.array([[-2.72496662e+01,  7.49222183e+00, -2.59609669e-02,  2.48445183e-01,
  -5.24457216e-01, -4.99831624e-02, -3.78916770e-01, -6.54165149e-01,
  -4.86877322e-01, -4.90657613e-02, -5.08075476e-01, -8.81507337e-01,
  -5.23021594e-02],
 [-2.75699940e+01,  8.75121689e+00,  1.03274250e+00,  1.16922021e+00,
   7.82192707e-01,  8.67679954e-01, -6.86519504e-01, -4.94599551e-01,
  -7.16070533e-01, -9.64982584e-02, -1.89103052e-01, -5.67111433e-01,
   2.26660743e-01],
 [-2.76201630e+01,  8.51478481e+00,  2.45062917e-01,  1.10437202e+00,
   1.32302594e+00,  1.15109706e+00, -5.53469360e-01, -1.59685218e+00,
  -3.86338055e-01, -1.00724295e-01, -2.13401064e-01, -8.73005927e-01,
  -7.13057593e-02],
 [-2.88381729e+01,  7.91480684e+00,  6.70008183e-01,  1.49881768e+00,
   1.73428714e+00,  8.56945634e-01, -1.86938837e-01, -9.24983025e-01,
   8.01171288e-02,  1.24482238e+00,  1.05534458e+00,  5.09452462e-01,
   5.07021010e-01],
 [-2.81988335e+01,  9.04975700e+00,  1.29980171e+00,  5.32774627e-01,
  -6.44544438e-02,  4.68814611e-01, -1.68238893e-01, -4.71396893e-01,
   1.93558499e-01,  1.15171087e+00,  8.09038281e-02, -7.97770858e-01,
   5.65259933e-01],
 [-2.71711082e+01,  9.01874256e+00,  9.56650853e-01,  1.59221399e+00,
   1.04040051e+00,  6.65737927e-01, -1.27099697e-02, -1.65264264e-01,
   1.33498937e-01,  1.13712204e+00, -2.32317269e-01, -2.28960849e-02,
   5.99904180e-01],
 [-2.85027962e+01,  8.15333080e+00,  1.42285240e+00,  1.67495108e+00,
   7.17887282e-01, -1.58032775e-01, -3.65235694e-02, -3.67568225e-01,
   9.71786737e-01,  8.50276649e-01, -1.00120211e+00, -7.23317325e-01,
  -1.49773300e-01],
 [-2.76931076e+01,  9.19399834e+00,  1.44285393e+00,  1.76776421e+00,
   1.01064432e+00,  4.18897778e-01,  5.00365674e-01, -2.30275225e-02,
   4.51116085e-01,  8.37219298e-01, -2.88589209e-01, -4.83666599e-01,
   4.39330578e-01],
 [-2.79189739e+01,  9.29881287e+00,  9.34956014e-01,  1.20677018e+00,
   5.31854093e-01,  6.49120331e-01, -6.94429159e-01, -6.16733968e-01,
   4.82238233e-01,  1.28356981e+00,  3.87966126e-01, -1.42310441e-01,
   6.21684015e-01],
 [-2.67068748e+01,  8.65606308e+00,  9.84331667e-01,  1.81623936e+00,
   6.02442861e-01, -6.15358129e-02,  3.29223752e-01, -3.64306360e-01,
  -4.42056835e-01,  4.64927942e-01,  4.71348196e-01, -2.02443615e-01,
   2.08276436e-01],
 [-2.49328766e+01,  9.94923306e+00,  2.50264466e-01,  2.94631481e+00,
   5.60430169e-01, -1.11898339e+00, -4.11021352e-01, -1.51185095e+00,
  -6.46494329e-01,  1.43851972e+00,  2.41595637e-02, -3.52725953e-01,
  -4.74318355e-01],
 [-2.68460331e+01,  1.00761337e+01,  1.35195720e+00,  1.23718297e+00,
   2.57312715e-01, -2.02319935e-01, -8.40575337e-01, -1.04642177e+00,
  -5.87131023e-01,  1.20236588e+00,  4.86623449e-03, -1.22079313e-01,
  -7.34620094e-02],
 [-2.74750423e+01,  9.64040661e+00,  1.72365737e+00,  1.55617225e+00,
   4.72396523e-01, -2.41079137e-01, -4.95002180e-01, -2.47908562e-01,
   1.06695461e+00,  1.13035750e+00,  2.20676646e-01,  2.05153719e-01,
   1.74277470e-01],
 [-2.80457249e+01,  9.16175842e+00,  1.43761337e+00,  1.02280343e+00,
   4.28281844e-01,  6.93321288e-01,  3.68839055e-01, -2.96375275e-01,
   3.02701801e-01,  9.37876046e-01,  7.59708583e-01,  3.32585186e-01,
   2.55709201e-01],
 [-2.74222298e+01,  8.62062168e+00,  1.02442181e+00,  6.91430867e-01,
   1.09218486e-01, -2.96385944e-01, -5.88724494e-01, -6.47571504e-01,
   5.65419972e-01,  9.38435435e-01, -1.67162329e-01, -6.47202671e-01,
  -4.19440508e-01],
 [-2.72814484e+01,  8.08693886e+00,  1.29827857e+00,  1.33874810e+00,
   7.34463394e-01,  2.23953594e-02, -8.78913522e-01, -1.37542260e+00,
  -8.09814110e-02,  9.81252015e-01, -1.00530110e-01, -5.73551595e-01,
  -6.88799977e-01],
 [-2.79454803e+01,  7.35683250e+00, -3.56415540e-01,  6.47932649e-01,
   3.48920166e-01, -6.61833882e-01, -1.35143816e+00, -5.95087409e-01,
   2.35486746e-01,  7.77948022e-01,  3.81046563e-01, -1.59318522e-01,
   4.42011580e-02],
 [-2.62574177e+01,  8.78383160e+00, -4.18078154e-01,  9.52693284e-01,
   1.35193691e-01,  2.26415515e-01, -8.84689331e-01, -1.33630157e-01,
  -1.17644228e-01,  1.04223394e+00,  8.46285880e-01,  2.18778670e-01,
   5.13047099e-01],
 [-2.57897282e+01,  9.96648979e+00,  1.97173610e-01,  8.33774805e-01,
   7.30423629e-01, -2.81709790e-01,  1.01190731e-02,  1.73441052e-01,
   1.47772849e-01,  3.41917515e-01, -8.95570517e-01, -2.56108761e-01,
   2.88692445e-01],
 [-2.66265488e+01,  8.80946445e+00,  3.58406574e-01,  6.34955704e-01,
   6.69554889e-01,  1.29825270e+00,  6.09130442e-01,  4.95338216e-02,
  -7.43597686e-01,  5.90681672e-01, -8.13212693e-02,  7.88209736e-02,
  -2.30369717e-01],
 [-2.43678341e+01,  8.68620872e+00, -5.45803130e-01,  2.35830045e+00,
   4.92700547e-01, -7.05754638e-01,  7.00326622e-01,  1.26533717e-01,
  -7.48960316e-01,  3.93145233e-01,  1.91998303e-01, -5.25702119e-01,
   4.02585715e-01],
 [-2.16801510e+01,  7.67340851e+00, -2.26567173e+00,  4.85116148e+00,
   4.51454818e-01, -3.08870125e+00,  5.69052160e-01, -6.69207513e-01,
  -1.87180841e+00,  2.36356810e-01, -9.80702877e-01,  6.75238192e-01,
   5.42292893e-01],
 [-2.19418182e+01,  7.26204872e+00, -2.61943913e+00,  6.42642879e+00,
   2.49493897e-01, -2.59476876e+00,  1.19680190e+00, -1.45224094e+00,
  -2.12063074e+00, -4.31247830e-01, -1.11739457e+00, -7.84448534e-02,
  -4.12257612e-02],
 [-1.96454544e+01,  7.43887234e+00, -2.30549145e+00,  7.68130875e+00,
  -3.14510465e-02, -3.81593156e+00,  1.06197965e+00, -1.94676089e+00,
  -1.55988812e+00, -9.33570191e-02, -9.70398366e-01,  4.08528864e-01,
  -2.63110667e-01],
 [-1.78843536e+01,  8.07788277e+00, -2.88101792e+00,  7.87806654e+00,
  -3.43052804e-01, -4.26013517e+00,  1.73972082e+00, -1.76900256e+00,
  -1.47710609e+00,  8.60049844e-01, -6.37780905e-01,  1.83025688e-01,
  -6.71735525e-01],
 [-1.84438057e+01,  6.60221434e+00, -3.51806593e+00,  6.67058754e+00,
  -1.42264116e+00, -5.31173944e+00,  1.10625935e+00, -1.40062571e+00,
  -2.00188422e+00,  3.40241522e-01, -8.00305068e-01,  8.82494688e-01,
   5.29406965e-01],
 [-1.83280239e+01,  7.34146976e+00, -2.54495358e+00,  8.08241272e+00,
  -1.07607579e+00, -4.89487696e+00,  1.56617045e+00, -1.02721095e+00,
  -1.51383293e+00,  2.83045205e-03, -7.21026719e-01,  9.70919907e-01,
   6.84914231e-01],
 [-1.79092789e+01,  7.46157598e+00, -3.59804010e+00,  7.86996174e+00,
  -6.49088442e-01, -5.00560379e+00,  8.47687662e-01, -1.20538342e+00,
  -1.21787739e+00,  5.87524831e-01, -4.62055773e-01,  1.53040183e+00,
   6.21838212e-01],
 [-1.80475559e+01,  7.81303787e+00, -3.46335435e+00,  8.21556568e+00,
  -9.76635516e-01, -5.11735439e+00,  1.61190480e-01, -1.09904456e+00,
  -4.99605209e-01,  9.60200727e-01, -6.77262425e-01,  1.35093629e+00,
   6.51066542e-01],
 [-1.75497894e+01,  8.11841393e+00, -4.38468981e+00,  7.11034679e+00,
  -1.48753333e+00, -4.96570826e+00,  1.11520123e-02, -1.74166393e+00,
  -9.95929539e-01,  1.12337458e+00, -7.69516230e-01,  6.64264619e-01,
   5.60940742e-01],
 [-1.73709240e+01,  8.26812077e+00, -3.59992528e+00,  6.42712116e+00,
  -1.15695286e+00, -5.10094976e+00, -5.79804122e-01, -1.43240905e+00,
   7.84628153e-01,  2.50789690e+00, -3.21808755e-01,  5.15936315e-01,
  -1.23884819e-01],
 [-1.80092793e+01,  1.00414677e+01, -2.78716660e+00,  4.22148085e+00,
  -1.78273058e+00, -4.57015753e+00, -4.20051217e-01, -8.10064077e-01,
   1.04422081e+00,  2.36305571e+00, -1.02415413e-01, -2.83852965e-02,
   1.30283415e-01],
 [-1.98658829e+01,  9.46356106e+00, -2.60526609e+00,  2.90001988e+00,
  -1.43236649e+00, -3.83263969e+00, -5.49135447e-01, -9.40321088e-01,
   2.09811896e-01,  2.46546626e+00, -3.18658561e-01, -5.22837162e-01,
  -2.10954860e-01],
 [-2.11215820e+01,  1.06815004e+01, -4.08981442e-01,  2.93189263e+00,
  -9.04713869e-01, -3.08265901e+00, -9.91533637e-01, -1.02585495e+00,
  -1.81448579e-01,  2.08046317e+00,  2.15604395e-01,  1.87991902e-01,
   4.24613148e-01],
 [-2.55553150e+01,  9.55604458e+00,  6.85233474e-01,  2.63482857e+00,
   7.24789739e-01, -5.93799472e-01, -3.79321933e-01, -2.16643110e-01,
   1.57414138e-01,  1.38627839e+00,  4.61592525e-01, -1.40175819e-01,
   2.49037862e-01],
 [-2.68548489e+01,  9.31414986e+00,  1.40447402e+00,  1.60748863e+00,
   7.56659150e-01, -3.02002672e-02, -2.56571591e-01, -6.96430504e-01,
  -4.31368098e-04,  1.22624385e+00,  6.98834658e-01, -6.45354331e-01,
   7.18588606e-02],
 [-2.73629589e+01,  8.36104488e+00,  3.91553253e-01,  1.06927228e+00,
  -3.85548979e-01,  2.48305351e-01,  4.26639408e-01, -8.25938642e-01,
   7.92732313e-02,  1.99990714e+00,  3.62807930e-01, -1.28722286e+00,
  -3.98830116e-01],
 [-2.61130486e+01,  7.56059170e+00,  1.47582710e-01,  3.44637632e+00,
   6.79924726e-01, -3.45066071e-01,  7.71066546e-01, -1.21190989e+00,
  -7.15058804e-01,  1.11889589e+00,  1.87315047e-01,  5.68902254e-01,
   1.12767100e+00],
 [-2.80662975e+01,  8.92466164e+00,  1.13411117e+00,  1.47177017e+00,
   8.15716505e-01,  7.48573482e-01,  8.05534661e-01,  7.35668778e-01,
   1.53028458e-01,  6.43611014e-01, -5.17833352e-01, -2.61508435e-01,
   8.92646015e-02],
 [-2.73665848e+01,  9.52764797e+00,  1.02254534e+00,  1.36045420e+00,
   6.60794258e-01, -9.83757675e-02, -1.01816162e-01,  7.23930299e-02,
  -4.90055382e-01,  2.81756163e-01, -4.25227016e-01, -2.71866798e-01,
  -3.19437355e-01],
 [-2.76010685e+01,  9.19162083e+00,  1.76603651e+00,  1.15300655e+00,
   9.55569386e-01,  1.06627762e+00, -4.34699021e-02,  3.40978086e-01,
   2.15051070e-01,  4.60923523e-01, -4.11799610e-01, -8.81207943e-01,
  -3.96679103e-01],
 [-2.85070591e+01,  7.91414928e+00,  1.02387726e+00,  8.13102007e-01,
   1.54520333e+00,  1.16286373e+00, -5.31589687e-01, -1.55581459e-01,
   8.22796404e-01,  1.27797461e+00,  1.72537565e-01, -5.76664209e-01,
  -3.70409071e-01],
 [-2.89601383e+01,  7.40117931e+00, -2.35922024e-01,  9.34813201e-01,
   1.83812642e+00,  1.49795413e+00, -3.41974288e-01, -4.19809282e-01,
   3.67597461e-01,  1.15231490e+00, -3.21705937e-01, -9.28833902e-01,
  -6.82076037e-01],
 [-2.77869549e+01,  8.11128044e+00,  1.23275793e+00,  1.31971431e+00,
   1.58573306e+00,  1.41767836e+00,  4.63125765e-01,  5.72615899e-02,
   2.74127275e-01,  5.05498657e-03, -1.13977563e+00, -1.59491396e+00,
  -4.96348858e-01],
 [-2.85782852e+01,  7.95060873e+00,  1.24929321e+00,  1.14951646e+00,
   7.20668674e-01,  9.83528793e-01, -1.62406564e-01, -3.06091726e-01,
   5.21706283e-01,  1.13171816e+00,  4.47053015e-01, -4.14752334e-01,
  -6.30317554e-02],
 [-2.80128288e+01,  7.58836174e+00,  1.57255840e+00,  2.29020429e+00,
   6.03333414e-01,  7.14710236e-01, -8.65851864e-02, -2.85091519e-01,
   4.88890439e-01,  1.15282393e+00,  2.12532818e-01, -5.73508143e-01,
  -2.34479845e-01],
 [-2.70119991e+01,  7.93215418e+00,  2.02548504e+00,  2.48576140e+00,
   1.40098953e+00,  3.48690301e-01, -4.61082608e-01, -8.74448895e-01,
   4.37710166e-01,  1.16959548e+00,  5.05856037e-01, -4.06073689e-01,
  -9.31890085e-02],
 [-2.65818424e+01,  9.02089882e+00,  1.70108140e+00,  8.95068705e-01,
   3.06215227e-01,  3.00291218e-02, -1.01846969e+00, -9.24321592e-01,
  -1.10439956e+00,  2.61868387e-01, -6.09666370e-02, -1.10956466e+00,
   5.10931849e-01],
 [-2.86718559e+01,  8.04953766e+00,  1.24920130e+00,  8.06950748e-01,
   1.60895097e+00,  1.07791817e+00, -4.77029523e-03,  1.82731509e-01,
  -4.36948568e-01,  1.55956790e-01,  9.07425880e-02, -6.79361582e-01,
  -7.22270608e-01]], dtype=np.float32)

    # Verifica que el array tiene forma (49, 13)
    if sample_float32.shape != (49, 13):
        raise ValueError("La muestra debe tener 49 filas y 13 columnas")

    # Añadir la dimensión de canal: de (49, 13) a (49, 13, 1)
    sample_float32 = np.expand_dims(sample_float32, axis=-1)

    # Añadir la dimensión de batch: de (49, 13, 1) a (1, 49, 13, 1)
    sample_float32 = np.expand_dims(sample_float32, axis=0)

    yield [sample_float32]

model = tf.keras.models.load_model(MODEL_H5_PATH)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]

tflite_model = converter.convert()
with open(TFLITE_PATH, "wb") as f:
    f.write(tflite_model)

print("Modelo cuantizado")

INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmpj1h896o9\assets


INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmpj1h896o9\assets


Modelo cuantizado


In [74]:
TFLITE_PATH = "../models/compressed_MFCC/model_CNNA.tflite"
interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Tipo de entrada: {input_details[0]['dtype']} ")
print(f"Tipo de salida: {output_details[0]['dtype']} ")
print("Formato de entrada esperado:", model.input_shape)

ops_set = set()
for idx in range(len(interpreter._get_ops_details())):
    op_name = interpreter._get_ops_details()[idx]['op_name']
    ops_set.add(op_name)

print("\nOperaciones utilizadas en el modelo TFLite:")
for op in sorted(ops_set):
    print(f"{op}")


Tipo de entrada: <class 'numpy.float32'> 
Tipo de salida: <class 'numpy.float32'> 
Formato de entrada esperado: (None, 49, 13)

Operaciones utilizadas en el modelo TFLite:
ADD
CONV_2D
FULLY_CONNECTED
MAX_POOL_2D
MUL
PACK
RESHAPE
SHAPE
SOFTMAX
STRIDED_SLICE


In [45]:
# Rutas
TFLITE_PATH = "../models/compressed_MFCC/model_CNN.tflite"
CPP_PATH = "../V_MFCC/main/model.cpp"
HEADER_PATH = "../V_MFCC/main/model.hpp"

# Cargar modelo TFLite cuantizado
with open(TFLITE_PATH, "rb") as f:
    tflite_model = f.read()

#Convertir a array de bytes en formato C++
cpp_array = ", ".join(str(b) for b in tflite_model)

# 📄 Generar código C++ (`model.cpp`)
cpp_code = f"""#include "model.hpp"

alignas(8) const unsigned char model_tflite[] = {{
    {cpp_array}
}};

const int model_tflite_len = {len(tflite_model)};
"""

# Guardar `model.cpp`
with open(CPP_PATH, "w") as f:
    f.write(cpp_code)

# 📄 Generar código C++ (`model.h`)
header_code = """#ifndef MODEL_H_
#define MODEL_H_

extern const unsigned char model_tflite[];
extern const int model_tflite_len;

#endif  // MODEL_H_
"""

# Guardar `model.h`
with open(HEADER_PATH, "w") as f:
    f.write(header_code)

print(f"Modelo exportado a C++ en {CPP_PATH}")
print(f"Header guardado en {HEADER_PATH}")

Modelo exportado a C++ en ../V_MFCC/main/model.cpp
Header guardado en ../V_MFCC/main/model.hpp


Código para comparar tamaños

In [31]:
def model_size(n_bytes):
    for unit in ['B','KB','MB','GB','TB']:
        if n_bytes < 1024.0:
            return f"{n_bytes:.5f} {unit}"
        n_bytes /= 1024.0

# Rutas a tus modelos
keras_path = "../models/not_compressed_audio/model_NN.h5"
tflite_int8_path = "../models/compressed_int8/model_NN.tflite"
tflite_float_path = "../models/compressed_float32/model_NN.tflite"

size_keras  = os.path.getsize(keras_path)
size_tflite_int8 = os.path.getsize(tflite_int8_path)
size_tflite_float = os.path.getsize(tflite_float_path)

print("Keras model:", model_size(size_keras))
print("TFLite int model:", model_size(size_tflite_int8))
print("TFLite float model:", model_size(size_tflite_float))

Keras model: 5.89761 MB
TFLite int model: 503.15625 KB
TFLite float model: 1.95738 MB
